## Load

In [1]:
import json

In [2]:
with open("raw/ltl.txt", "r") as f:
    raw_ltls = f.read().splitlines()
    unique_ltls = set(raw_ltls)
with open("raw/eng.txt", 'r') as f:
    raw_engs = f.read().splitlines()

DPs = []
for ltl, eng in zip(raw_ltls, raw_engs):
    DPs.append({'ltl': ltl, 'eng': eng})

In [3]:
for ltl in sorted(unique_ltls, key=lambda x: len(x)):
    print(ltl)

F B
F C
F Y
F R
F & R F Y
F & R F Z
F & C F B
F & R F X
F & R F C
F & Y F B
F & C F Y
F & B F R
F & B F C
F & R F B
F & C F R
F & B F Y
F & Y F C
& F C G ! R
& F Y G ! C
& F R G ! C
& F Y G ! B
& F R G ! B
& F C G ! Y
& F B G ! R
& F B G ! Y
& F Y G ! R
& F C G ! B
& F B G ! C
& F R G ! Y
F & | R B F Y
F & | R Y F B
F & | Y B F C
F & | R B F C
F & | C Y F B
F & | R Y F C
F & | R C F B
F & | C Y F R
F & | C R F B
F & | B Y F C


In [4]:
print("Clean-up Domain, with augmentation")
print("Number of Data Points", len(DPs))
print("Number of unique LTL expressions:", len(unique_ltls))
print("Number of unique LTL structures:", 4)

Clean-up Domain, with augmentation
Number of Data Points 3382
Number of unique LTL expressions: 39
Number of unique LTL structures: 4


In [5]:
seen_length = set()
for dp in DPs:
    if len(dp['ltl']) not in seen_length:
        seen_length.add(len(dp['ltl']))
        print(dp['ltl'])
        print(dp['eng'])
        print()

F & B F C
go to the blue room keep going and stop when you reach the green room

F R
move to the red room

& F C G ! Y
go only through rooms that are not yellow to get to the green room

F & | R Y F B
go through the yellow or red room to reach the blue room



## Clean up LTL expressions 

In [6]:
Room_Types = [('B', 'go to the blue room'), ('R', 'go to the red room'), ('Y', 'go to the yellow room'), ('C', 'go to the green room')]

In [7]:
APs = [
    {'ap': 'B', 'eng': 'go to the blue room'},
    {'ap': 'R', 'eng': 'go to the red room'},
    {'ap': 'Y', 'eng': 'go to the yellow room'},
    {'ap': 'C', 'eng': 'go to the green room'},
]

In [8]:
def build_type_A(ap1):
    return {
        'raw_ltl': f"F {ap1['ap']}",
        'canonical_ltl': f"finally {ap1['eng']}",
        'eng': f"{ap1['eng']}"}

def build_type_B(room_1, room_2):
    return {
        "raw_ltl": f"F & {room_1['ap']} F {room_2['ap']}",
        "canonical_ltl": f"finally ( and (  {room_1['eng']} , finally ( {room_2['eng']} )  )  )",
        "eng": f"{room_1['eng']} first, and then {room_2['eng']}"}

def build_type_C(room_1, room_2):
    return {
        "raw_ltl": f"& F {room_1['ap']} G ! {room_2['ap']}",
        "canonical_ltl": f"and ( finally ( {room_1['eng']} ) , globally ( not ( {room_2['eng']} ) ) )",
        "eng": f"{room_1['eng']}, and do not ever {room_2['eng']}"
    }

def build_type_D(room_1, room_2, room_3):
    return {
        "raw_ltl": f"F & | {room_1['ap']} {room_2['ap']} F {room_3['ap']}",
        "canonical_ltl": f"finally ( and ( or ( {room_1['eng']} , {room_2['eng']} ) , finally ( {room_3['eng']} ) ) )",
        "eng": f"{room_1['eng']} or {room_2['eng']} to finally {room_3['eng']}"
    }

In [9]:
translation_seeds = []
for r1 in APs:
    translation_seeds.append(build_type_A(r1))
    for r2 in APs:
        if r1 == r2:
            continue
        translation_seeds.append(build_type_B(r1, r2))
        translation_seeds.append(build_type_C(r1, r2))
        for r3 in APs:
            if r1 == r3 or r2 == r3:
                continue
            translation_seeds.append(build_type_D(r1, r2, r3))

In [10]:
print(len(translation_seeds))

52


In [11]:
seed_ltls = set([t['raw_ltl'] for t in translation_seeds])
print(unique_ltls - seed_ltls)

{'F & R F Z', 'F & R F X'}


In [12]:
additional_seed_1 = {
    "raw_ltl": "F & R F X",
    "canonical_ltl": "finally ( and (  go to the red room , finally ( go to the blue room with chair )  )  )",
    "eng": "go to the red room and push the chair into the blue room"
}

additional_seed_2 = {
    "raw_ltl": "F & R F Z",
    "canonical_ltl": "finally ( and (  go to the red room , finally ( go to the green room with chair )  )  )",
    "eng": "go to the red room and push the chair into the green room"
}

translation_seeds.append(additional_seed_1)
translation_seeds.append(additional_seed_2)

In [13]:
len(translation_seeds)

54

In [14]:
seed_ltls = set([t['raw_ltl'] for t in translation_seeds])
print(unique_ltls - seed_ltls)

set()


### Save the canonical decoding list

In [15]:
possible_decodings = {}
for seed in translation_seeds:
    canonical = seed['canonical_ltl']
    possible_decodings[canonical] = {
        'formula': canonical,
        'raw': seed['raw_ltl'],
    }

In [16]:
with open("canonical.json", 'w') as f:
    f.write(json.dumps(possible_decodings, indent=2))

### Save translation seed for zero shot

In [17]:
with open("train_seed.jsonl", "w") as f:
    for dp in translation_seeds:
        better_ltl = dp['canonical_ltl']
        entry = {'canonical': better_ltl, 'formula': better_ltl, 'natural': dp['eng']}
        json.dump(entry, f)
        f.write('\n')

### Save golden data for evaluation  

In [18]:
raw_canonical_mapping = {
    seed['raw_ltl']: seed['canonical_ltl'] for seed in translation_seeds
}

In [19]:
len(DPs)

3382

In [20]:
DPs[0]

{'ltl': 'F & B F C',
 'eng': 'go to the blue room keep going and stop when you reach the green room'}

In [21]:
with open("golden.jsonl", "w") as f:
    for dp in DPs:
        entry = {
            'canonical': raw_canonical_mapping[dp['ltl']],
            'formula': raw_canonical_mapping[dp['ltl']],
            'natural': dp['eng'],
            'raw_ltl': dp['ltl'],
        }
        json.dump(entry, f)
        f.write('\n')